# Prep Data

In [1]:
import numpy as np 
import pandas as pd 
import os
import json
import nltk # NLP  
from nltk.corpus import stopwords  
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer 
import re  
import contractions

In [2]:
%cd datasets

/Users/aij/Documents/2023 Sem 2/GENG5512/wills-ensemble/datasets


In [3]:
with open('twitter.txt', 'r') as f:
    lines = f.readlines()

with open('twitter15.txt', 'r') as f:
    t15_lines = f.readlines()

with open('twitter15_label.txt', 'r') as f:
    t15_labels = f.readlines()

with open('twitter16.txt', 'r') as f:
    t16_lines = f.readlines()

with open('twitter16_label.txt', 'r') as f:
    t16_labels = f.readlines()

['731166399389962242\t🔥ca kkk grand wizard 🔥 endorses @hillaryclinton #neverhillary #trump2016 URL\n', '714598641827246081\tan open letter to trump voters from his top strategist-turned-defector URL via @xojanedotcom\n', '691809004356501505\t"america is a nation of second chances" —@potus on new reforms to solitary confinement: URL URL\n', '693204708933160960\tbrandon marshall visits and offers advice, support to brother of fallen hero zaevion dobson: URL URL\n', '551099691702956032\trip elly may clampett: so sad to learn #beverlyhillbillies star donna douglas has passed away. URL\n', '767223774072676354\tformer 3 doors down guitarist matt roberts has died at age 38, according to his father. URL URL\n', '715515982584881152\tcraigslist ad: ‘get paid $15 an hour to protest at the trump rally’ - URL URL\n', '514106273852174337\tjust in: missing afghan soldiers found trying to enter canada near niagara falls URL URL\n', '500319801344929795\tthe day #ferguson cops told a dirty, bloody lie (

In [4]:
twitter_df = pd.DataFrame(columns=["text", "verdict"])
for line in lines:
    sections = line.split('\t')
    verdict = sections[2][:-1]
    if verdict == "non-rumor":
        continue
    if verdict == "unverified":
        verdict = "false"

    data = [sections[1], verdict]
    twitter_df.loc[len(twitter_df)] = data
print(twitter_df)

                                                   text verdict
0     ohio police officer hospitalized after eating ...   false
1     one of the men who wanted to copy the #orlando...   false
2     missouri governor declares state of emergency ...   false
3     the iphone 6 plus really does have a bending p...   false
4     paul walker dies, millions cried. millions die...    true
...                                                 ...     ...
1724  kim davis angers 'eye of the tiger' creator af...   false
1725  ever wonder why the most mass shootings ever h...   false
1726  #breaking hostages held inside sydney cafe, is...    true
1727  newsflash⚡️the #orlando shooter was a muslim a...   false
1728  great scott! #backtothefuture 2 (almost) naile...   false

[1729 rows x 2 columns]


In [7]:
twitter16_df = pd.DataFrame(columns=["text", "verdict"])
for i, line in enumerate(t16_lines):
    sections = line.split('\t')
    labels = t16_labels[i].split(':')
    verdict = labels[0]
    #rint(verdict)
    if verdict == "false" or verdict == "unverified":
        verdict = "true"
    if verdict == "non-rumor":
        verdict = "false"
    # print(sections[1])
    data = [sections[1], verdict]
    # print(data)
    twitter16_df.loc[len(twitter16_df)] = data
print(twitter16_df)

                                                  text verdict
0    correct predictions in back to the future ii U...    true
1    .@whitehouse in rainbow colors for #scotusmarr...    true
2    cops bought the alleged church shooter burger ...    true
3    🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...    true
4    an open letter to trump voters from his top st...    true
..                                                 ...     ...
813  jeb bush campaign kicks off 3-state farewell t...   false
814  breaking: live coverage of hostage situation u...    true
815                  “after school satan clubs”? URL\n    true
816  this network of tunnels is from the stone age ...    true
817  top north korean diplomat who disappeared from...   false

[818 rows x 2 columns]


In [30]:
# download nltk stuff
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/aij/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/aij/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/aij/nltk_data...


True

In [31]:
# Clean text

def remove_punctuations(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_characters(text):
    return re.sub('[^a-zA-Z]', ' ', text)

def remove_url(text):
    return re.sub('URL', '', text)

def remove_contractions(text):
    return ' '.join([contractions.fix(word) for word in text.split()])

stop_words = stopwords.words('english')
def remove_stopwords(text):
    return ' '.join([word for word in nltk.word_tokenize(text) if word not in stop_words])
# str(text).split()

#defining the object for Lemmatization
lemmatizer = WordNetLemmatizer()
#defining a function for lemming
def lemmatize_words(text):
    return ' '.join(lemmatizer.lemmatize(word) for word in text.split())

#defining the object for stemming
stemmer = PorterStemmer()
#defining a function for stemming
def stemming_words(text):
    return ' '.join(stemmer.stem(word) for word in text.split())

def clean_text(text):
    text = remove_url(text)
    text = remove_contractions(text)
    text = text.lower()
    text = remove_punctuations(text)
    text = remove_characters(text)
    text = remove_stopwords(text)
    text = stemming_words(text)
    text = lemmatize_words(text)
    return text
    
# apply
twitter_df['text'] = twitter_df['text'].apply(clean_text)
twitter_df["text"].values

twitter15_df['text'] = twitter15_df['text'].apply(clean_text)
twitter15_df["text"].values

array(['ca kkk grand wizard endors hillaryclinton neverhillari trump',
       'open letter trump voter top strategist turn defector via xojanedotcom',
       'america nation second chanc potu new reform solitari confin', ...,
       'break news accord document releas press mikebrown connect earlier robberi ferguson',
       'ebola vaccin news today',
       'concern airport passeng suit homemad hazmat flight'], dtype=object)

In [32]:
print(twitter15_df.shape)
twitter15_df.to_csv("twitter15.csv")

(1490, 2)
